---
title: "Directional CRNS measurements"
output: html_notebook
---

# Illustrate theoretical discernability and accuracy of CRNS signal of a directional CRNS probe
Scripts for generating the figures of the manuscript
"Assessing the feasibility of a directional CRNS-sensor for estimating soil moisture"
by Till Francke, Maik Heistermann, Markus Köhli, Christian Budach, Martin Schrön, Sascha E. Oswald

## 0. parameters ####
This chunk contains the parameters of the simulation, or lets you choose the pre-defined scenarios "bestcase" and "worstcase".


## 1. simulate  for a single setting ####

This chunk does the computations for the following two plots.

## 1.1 plot counts and CIs (fig. 7 / ci_example) ####

Example for 95 \%-confidence intervals for the count rates obtained within the half-spaces (${R}_\text{1,total}$ = 2100 counts/h, ${R}_\text{2,total}$ = 2520 counts/h, "bestcase scenario", moderate contrast $\Delta R=0.2$), by the directional CRNS-sensors (${R}_\text{f1,total}$, ${R}_\text{f2,total}$) and the rates reconstructed from the directional sensors (${R}_\text{1r,total}$, ${R}_\text{2r, total}$). The vertical arrows indicate the minimum required aggregation time to confine the CI within 5 \% of the true value ($\Delta t_\text{min}^{\text{determ}}$).

## 1.2 p-values of comparison (fig. 8 / p_example)####

p-values for discriminating pairs of measured count rates (${R}_\text {1,total}$ = 2100 counts/h, ${R}_\text {2,total}$ = 2520  counts/h, "bestcase scenario", moderate contrast $\Delta R=0.2$). The vertical arrows indicate the minimum required aggregation time $\Delta t_\text{min}^{\text{disting}}$ to statistically \emph{distinguish} two rates at p = 0.05.

# Simulations for a range of settings 
## 2.1 simulate  for a range of settings: R vs. t_min (CI-thresh) ####


## 2.1 plot results (fig. 9) ####

Minimum aggregation time $\Delta t_\text{min}^{\text{determ}}$ required to obtain the reconstructed count rate ${R}_\text{1r}$ from the two directional measurements with a CI smaller than 5 \% of the actual value.

## 2.2 simulate for a range of settings: R vs. t_min (p-thresh) ####


## 2.2 plot results (fig. 10) ####

Minimum aggregation time $\Delta t_\text{min}^{\text{disting}}$ required to statistically separate the reconstructed count rates  ${R}_\text{1r}$ and ${R}_\text{2r}$ from the two directional measurements with p-value < 0.05.

## 3.1 simulate  for a range of settings: R vs. contrasts (CI-thresh) ####


## 3.1 plot results (fig. 11)####

Maximum possible contrast $\Delta R$ in reconstructing the count rates ${R}_\text{1r}$ from the two directional measurements with a CI smaller than 5 \% of the true value.

## 3.2 simulate  for a range of settings: R vs. contrasts (p-thresh) ####


## 3.2 plot results (fig. 12)####

Minimum contrast $\Delta R$ required to statistically separate the reconstructed count rates  ${R}_\text{1r}$ and ${R}_\text{2r}$ from the two directional measurements with p-value < 0.05.

## 4.1 simulate  for a range of settings: aggr_time vs. contrasts (CI-thresh) ####


## 4.1 plot results (fig. 13)####

Maximum possible contrast $\Delta R$ to allow the reconstruction of the count rate ${R}_\text{1r}$ from the two directional measurements with a CI smaller than 5 \% of the true value with the aggregation time $\Delta t$.

## 4.2 simulate  for a range of settings: R vs. contrasts (p-thresh) ####


## 4.2 plot results (fig. 14)####

Minimum contrast $\Delta {R}$ required to statistically separate the count rates  ${R}_\text{1r}$ and ${R}_\text{2r}$ reconstructed from the two directional measurements with a p-value < 0.05.